# Agent初探

In [ ]:
from langchain_openai import OpenAI
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import (
    initialize_agent,
    AgentType
)

llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0.6
)
tools = load_tools(['ddg-search', 'llm-math'], llm=llm)
agent = initialize_agent(
    tools, llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=5,
    verbose=True,
    return_intermediate_steps=False
)
print(agent.invoke('OpenAI开发者大会召开的年份除以4，最后得到的数字是多少？'))

In [ ]:
from langchain.load.dump import dumps

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)
response = agent('OpenAI开发者大会召开的年份除以4，最后得到的数字是多少？')
print(response["intermediate_steps"])

# 使用LangChain提供的dumps方法可以将字典转换成JSON字符串
print(dumps(response["intermediate_steps"], pretty=True))

# Agent类型

## Chat ReAct

In [ ]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import (
    initialize_agent,
    AgentType
)

llm = OpenAI(
    model_name='gpt-3.5-turbo-instruct',
    temperature=0
)

chat_llm = ChatOpenAI(temperature=0)

tools = load_tools(['ddg-search', 'llm-math'], llm=llm)
agent = initialize_agent(
    tools, chat_llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
print(agent.invoke('OpenAI开发者大会召开的年份除以4，最后得到的数字是多少？'))

## ReAct Document Store

In [ ]:
from langchain.llms import OpenAI
from langchain_community.docstore.wikipedia import Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

# 初始化文档存储
docstore = DocstoreExplorer(Wikipedia())

# 初始化Search和Lookup工具
tools = [
    Tool(
        name='Search',
        func=docstore.search,
        description='useful for when you need to ask with search',
    ),
    Tool(
        name='Lookup',
        func=docstore.lookup,
        description='useful for when you need to ask with lookup',
    ),
]

llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools, llm,
    agent=AgentType.REACT_DOCSTORE,
    verbose=True
)
print(agent.invoke('秦朝建立是在什么时间？'))

## Conversational

In [ ]:
import os
from langchain_openai import OpenAI
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import GoogleSerperAPIWrapper


# 使用GoogleSerperAPIWrapper需要先到Serper官网注册登录
# 然后设置以下环境变量为你的API key
os.environ['SERPER_API_KEY'] = 'xxxxxx'

search =  GoogleSerperAPIWrapper()
tools = [
    Tool(
        name='search',
        func=search.run,
        description='用于搜索当前事件'
    )
]
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

llm = ChatOpenAI(model_name='gpt-4o',temperature=0)
agent = initialize_agent(
    tools, llm,
    # CHAT_CONVERSATIONAL_REACT_DESCRIPTION用于聊天类型模型
    # CONVERSATIONAL_REACT_DESCRIPTION用于LLM补全模型
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)
agent.invoke(input='小张今年25岁。')
print(agent.invoke('秦朝建立是哪一年？秦朝建立时间加上小张的年龄等于多少？'))

## OpenAI Function

In [ ]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType


@tool
def get_word_length(word):
    """计算单词的长度"""
    return len(word)


tools = [get_word_length]
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

print(agent.invoke(
    'How many letters are there in Hangzhou and Beijing?'))

## Self-Ask With Search

In [ ]:
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

llm = ChatOpenAI(
    model_name='gpt-4o',
    temperature=0
)
search = GoogleSerperAPIWrapper()

# SELF_ASK_WITH_SEARCH类型的Agent只能设置一个工具
tools = [
    Tool(
        name='Intermediate Answer',
        func=search.run,
        description='useful for when you need to ask with search',
    )
]

agent = initialize_agent(
    tools, llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True
)

print(agent.invoke('2023年微软CEO出生于哪个国家？'))

## Structured Tool Chat

In [ ]:
# 在非Jupyter Notebooks环境下可以不用导入nest_asyncio
import nest_asyncio

from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_community.agent_toolkits.playwright.toolkit import (
    PlayWrightBrowserToolkit)
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser)

nest_asyncio.apply()

# 初始化工具集
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(
    async_browser=async_browser)
tools = toolkit.get_tools()

llm = ChatOpenAI(temperature=0.5)
agent_chain = initialize_agent(
    tools, llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

response = await agent_chain.ainvoke(
    'movie.douban.com/subject/1292063/ 网页的标题是什么？')
print(response)

In [ ]:
chat_history = MessagesPlaceholder(
    variable_name='chat_history'
)
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

agent = initialize_agent(
    tools, llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={
        'memory_prompts': [chat_history],
        'input_variables': ['input', 'agent_scratchpad', 'chat_history']
    },
    memory=memory,
)

## OpenAI Assistant

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name='langchain assistant',
    instructions='You are a personal math tutor. '
                 'Write and run code to answer math questions.',
    tools=[
        {'type': 'code_interpreter'},
        {'type': 'retrieval'},
        DuckDuckGoSearchRun()
    ],
    model='gpt-4o',
    as_agent=True
)
output = interpreter_assistant.invoke(
    {'content': "What's 10 - 4 raised to the 2.7"}
)

In [ ]:
from langchain.agents import AgentExecutor
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents.openai_assistant import OpenAIAssistantRunnable

agent = OpenAIAssistantRunnable.create_assistant(
    name='langchain assistant',
    instructions='You are a personal math tutor. '
                 'Write and run code to answer math questions. '
                 'You can also search the internet.',
    model='gpt-4o',
    tools=[{'type': 'code_interpreter'}],
    as_agent=True,
)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[DuckDuckGoSearchRun()]
)
output = agent_executor.invoke(
    {'content': "What's the weather in SF today divided by 2.7"}
)

# 自定义Tool

## 使用Tool对象

In [ ]:
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field, root_validator


search = SerpAPIWrapper()
search_tool = Tool.from_function(
    func=search.run,
    name='Search',
    description='useful for when you need to answer '
                'questions about current events',
)

class WordInput(BaseModel):
    word: str = Field(description='Word to calculate length')

    @root_validator
    def validate_parameter(cls, values):
        """用于对传递进来的函数的参数进行验证"""
        word = values['word']
        try:
            # 如果可以转换成数字，那么它就是非文字字符
            int(word)
            raise ValueError(
                'The word parameter type must be a literal character')
        except:
            return values

def get_word_length(word):
    return len(word)

word_tool = Tool(
        name='get_word_length',
        description='Calculate the length of a word',
        func=get_word_length,
        args_schema=WordInput
    )


tools = [search_tool, word_tool]

## 继承BaseTool

In [ ]:
from typing import Optional

from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()


class CustomSearchTool(BaseTool):
    name = 'custom_search'
    description = ('useful for when you need to '
                   'answer questions about current events')

    def _run(
            self, query: str,
            run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """调用工具"""
        return search.run(query)

    async def _arun(
            self, query: str,
            run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """异步调用工具"""
        raise NotImplementedError('custom_search does not support async')


tools = [CustomSearchTool()]

## 使用Tool装饰器

In [ ]:
from langchain.tools import tool

@tool
def get_word_length(word):
    """计算单词长度"""
    return len(word)

In [ ]:
class WordInput(BaseModel):
    word: str = Field(description='Word to calculate length')

@tool("word_handler", return_direct=True, args_schema=WordInput)
def get_word_length(word):
    """计算单词长度"""
    return len(word)

## Structured Tool

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import AgentType, initialize_agent

def plus(a: float, b: float):
    """计算两个数的和"""
    return a + b

plus_tool = StructuredTool.from_function(plus)
llm = ChatOpenAI(temperature=0)

agent = initialize_agent(
    [plus_tool], llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
print(agent.invoke('6加10等于多少？'))

In [ ]:
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools import BaseTool
from langchain_community.utilities import SerpAPIWrapper


class CustomSearchTool(BaseTool):
    name = 'custom_search'
    description = ('useful for when you need to '
                   'answer questions about current events')

    def _run(
            self,
            query: str,
            engine: str = 'google',
            gl: str = 'us',
            hl: str = 'en',
            run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """调用工具"""
        search_wrapper = SerpAPIWrapper(
            params={'engine': engine, 'gl': gl, 'hl': hl})
        return search_wrapper.run(query)

    async def _arun(
            self,
            query: str,
            engine: str = 'google',
            gl: str = 'us',
            hl: str = 'en',
            run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """异步调用工具"""
        raise NotImplementedError('custom_search does not support async')

## 异常处理

In [ ]:
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.tools.base import ToolException
from langchain.agents import AgentType, initialize_agent


def _handle_error(error: ToolException) -> str:
    return f'工具执行过程中发生以下错误：{error.args[0]}'


def search_tool1(s: str):
    raise ToolException('The search tool1 is not available.')


def search_tool2(s: str):
    raise ToolException('The search tool2 is not available.')


def search_tool3(s: str):
    raise ToolException('The search tool3 is not available.')


description = ('useful for when you need to '
               'answer questions about current events')
tools = [
    Tool.from_function(
        func=search_tool1,
        name='Search_tool1',
        description=description,
        handle_tool_error=True,
    ),
    Tool.from_function(
        func=search_tool2,
        name='Search_tool2',
        description=description,
        handle_tool_error=_handle_error,
    ),
    Tool.from_function(
        func=search_tool2,
        name='Search_tool3',
        description=description,
        handle_tool_error='工具3出错啦',
    )
]

agent = initialize_agent(
    tools,
    ChatOpenAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
agent.invoke('"晴天"的作者是谁？')

# 人工校验及输入

## 默认人工校验

In [ ]:
from langchain_community.tools import ShellTool
from langchain_community.callbacks import HumanApprovalCallbackHandler

tool = ShellTool(callbacks=[HumanApprovalCallbackHandler()])
tool.run('ls')

## 自定义用户审批

In [ ]:
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_community.callbacks import HumanApprovalCallbackHandler


def _should_check(serialized_obj: dict) -> bool:
    # 当工具名称为terminal时，需要进行审批
    return serialized_obj.get('name') == 'terminal'


def _approve(_input: str) -> bool:
    if _input == 'echo "Hello World"':
        return True
    msg = (
        '是否批准执行以下输入？'
        '回复Y或者yes为批准，回复其他内容则视为不批准。'
    )
    msg += '\n\n' + _input + '\n'
    resp = input(msg)
    return resp.lower() in ('yes', 'y')


callbacks = [
    HumanApprovalCallbackHandler(
        should_check=_should_check,
        approve=_approve
    )]

llm = ChatOpenAI(temperature=0)
tools = load_tools(['wikipedia', 'llm-math', 'terminal'], llm=llm)
agent = initialize_agent(
    tools, llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

res = agent.invoke('秦朝统一六国是哪一年？', callbacks=callbacks)
print(res)
res = agent.invoke('请列出当前目录下的文件', callbacks=callbacks)
print(res)

## 人工输入

In [ ]:
from langchain.agents import AgentType
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent

llm = ChatOpenAI(
    model_name='gpt-4o',
    temperature=0
)
tools = load_tools(['human', 'llm-math'], llm=llm)

agent = initialize_agent(
    tools, llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # 如果出现结果解析错误，可以将handle_parsing_errors设置为True
    handle_parsing_errors=True,
    verbose=True
)
print(agent.invoke(
    '请先让用户一次性输入两个值（用逗号隔开），然后对这两个值进行加法运算'
))

# Agent实际应用

## 结合向量存储使用Agent

In [ ]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.agents import AgentType, Tool, initialize_agent

loader = WebBaseLoader('https://****.ruff.**/docs/faq/')[　请参考链接8-2。]
docs = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0
)
texts = text_splitter.split_documents(loader.load())

ruff_db = Chroma.from_documents(
    texts,
    OpenAIEmbeddings(),
    collection_name='ruff'
)
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type='stuff',
    retriever=ruff_db.as_retriever()
)

tools = [
    Tool(
        name='Ruff QA System',
        func=chain.run,
        description='当你需要回答关于ruff（一个Python linter）的问题时，它很有用。'
        # 如果希望直接返回chain的结果，可以设置如下参数：
        # return_direct=True
    )
]

agent = initialize_agent(
    tools, ChatOpenAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
print(agent.invoke('什么是ruff？'))


## Fake Agent（虚构代理）

In [ ]:
from langchain.agents import AgentType
from langchain_community.llms import FakeListLLM
from langchain.agents import initialize_agent
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]

responses = [
    'Action: Python REPL\nAction Input: print(2 + 2)',
    'Final Answer: 4'
]

llm = FakeListLLM(responses=responses)

agent = initialize_agent(
    tools, llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
print(agent.invoke('whats 2 + 2'))

## 自定义Agent

In [ ]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import (
    Tool, AgentExecutor, BaseSingleActionAgent
)


class FakeAgent(BaseSingleActionAgent):
    """自定义虚构Agent """

    @property
    def input_keys(self):
        return ["input"]

    def plan(
            self,
            intermediate_steps: List[Tuple[AgentAction, str]],
            **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        # 如果之前搜索到了，就直接返回搜索到的答案
        # 当整个Agent执行完时，需要返回AgentFinish对象
        if intermediate_steps:
            return AgentFinish(
                return_values={'output': intermediate_steps[-1][-1]},
                log=''
            )
        # 当Agent还未执行完时，需要返回AgentAction对象，
        # 并指定下一个要执行的工具
        return AgentAction(
            tool='Search',
            tool_input=kwargs['input'],
            log=''
        )

    async def aplan(
            self,
            intermediate_steps: List[Tuple[AgentAction, str]],
            **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        if intermediate_steps:
            return AgentFinish(
                return_values={'output': intermediate_steps[-1][-1]},
                log=''
            )
        return AgentAction(
            tool='Search',
            tool_input=kwargs['input'],
            log=''
        )


search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name='Search',
        func=search.run,
        description='useful for when you need to '
                    'answer questions about current events'
    )
]

agent = FakeAgent()
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
print(agent_executor.invoke('秦朝统一六国是哪一年？'))

## 自定义LLM Agent

In [ ]:
import re
from typing import List, Union

from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import StringPromptTemplate
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.schema import (
    AgentAction, AgentFinish, OutputParserException
)
from langchain.agents import (
    Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
)

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name='Search',
        func=search.run,
        description='useful for when you need to '
                    'answer questions about current events'
    )
]

template = '''Answer the following questions as best you can,
but speaking as a pirate might speak.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer.
Use lots of "Arg"s

Question: {input}
{agent_scratchpad}'''


# 自定义PromptTemplate
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # 获取中间步骤的信息列表，并按特定的格式将其格式化成字符串
        intermediate_steps = kwargs.pop('intermediate_steps')
        thoughts = ''
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f'\nObservation: {observation}\nThought: '

        # 将格式化好的内容存入agent_scratchpad对应的值中
        kwargs['agent_scratchpad'] = thoughts

        # 将工具列表按照“名字:描述”的格式转换成字符串，
        # 并将其存入tools对应的值中
        kwargs['tools'] = '\n'.join([
            f"{tool.name}: {tool.description}" for tool in self.tools
        ])
        # 创建工具名称列表字符串，并将其存入tool_names对应的值中
        kwargs['tool_names'] = ', '.join(
            [tool.name for tool in self.tools]
        )
        return self.template.format(**kwargs)


prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=['input', 'intermediate_steps']
)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # 如果LLM的输出中有“Final Answer”，则表示当前任务已执行完毕
        # 需要返回AgentFinish
        if 'Final Answer:' in llm_output:
            return AgentFinish(
                return_values={
                    'output': llm_output.split('Final Answer:')[-1].strip()
                },
                log=llm_output,
            )
        # 解析动作和动作输入
        reg = (r'Action\s*\d*\s*:(.*?)\n'
               r'Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)')
        match = re.search(reg, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(
                f'Could not parse LLM output: `{llm_output}`')
        action = match.group(1).strip()
        action_input = match.group(2)

        # 返回后续需要执行的AgentAction，并设置要执行的工具和工具输入
        return AgentAction(
            tool=action,
            tool_input=action_input.strip(' ').strip('"'),
            log=llm_output
        )


output_parser = CustomOutputParser()

tool_names = [tool.name for tool in tools]
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    # 这个停止标志是Prompt中用来表示观测（Observation）开始的那个标志。
    # 如果没有设定这样的标志，则LLM可能会“幻想”出一个观测结果
    stop=['\nObservation:'],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)
agent_executor.invoke('秦朝统一六国是哪一年？')

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

template_with_history = '''Answer the following questions as best you can,
but speaking as a pirate might speak.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer.
Use lots of "Arg"s

Previous conversation history:
{chat_history}

Question: {input}
{agent_scratchpad}'''

prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    input_variables=['input', 'intermediate_steps', 'chat_history']
)

tool_names = [tool.name for tool in tools]
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt_with_history)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=['\nObservation:'],
    allowed_tools=tool_names
)

memory=ConversationBufferWindowMemory(
    k=2, memory_key='chat_history'
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
)

agent_executor.invoke('秦朝统一六国是哪一年？')

## 自定义MRKL Agent

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name='Search',
        func=search.run,
        description='useful for when you need to '
                    'answer questions about current events',
    )
]

prefix = '''Answer the following questions as best you can,
but speaking as a pirate might speak.
You have access to the following tools:
'''

suffix = '''Begin! Remember to speak as a pirate
when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}
'''

# 创建prompt
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=['input', 'agent_scratchpad']
)

# 实例化LLMChain
llm_chain = LLMChain(
    llm=OpenAI(temperature=0),
    prompt=prompt
)
tool_names = [tool.name for tool in tools]

# 实例化agent
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    allowed_tools=tool_names
)

# 实例化agent执行器
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)
agent_executor.invoke('秦朝统一六国是哪一年？')

## 自定义具有工具检索功能的Agent

In [ ]:
import re
from typing import Union, Callable

from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import LLMChain
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain_community.utilities import GoogleSerperAPIWrapper

# 创建搜索工具
search = GoogleSerperAPIWrapper()
search_tool = Tool(
    name='Search',
    func=search.run,
    description='useful for when you need to '
                'answer questions about current events',
)

# 创建虚构工具
def fake_func(inp):
    return 'foo'

fake_tools = [
    Tool(
        name=f'foo-{i}',
        func=fake_func,
        description=f'a silly function that you can use to '
                    f'get more information about the number {i}',
    )
    for i in range(99)
]

ALL_TOOLS = [search_tool] + fake_tools

docs = [
    Document(page_content=t.description, metadata={'index': i})
    for i, t in enumerate(ALL_TOOLS)
]

vector_store = Chroma.from_documents(
    docs,
    OpenAIEmbeddings(),
    collection_name='tools'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 2})

def get_tools(query):
    """通过查询获取最适合的工具"""
    _docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata['index']] for d in _docs]


template = '''Answer the following questions as best you can,
but speaking as a pirate might speak.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer.
Use lots of "Arg"s

Question: {input}
{agent_scratchpad}'''


class CustomPromptTemplate(StringPromptTemplate):
    template: str
    # 可用的工具列表
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # 获取中间步骤的信息列表，并按特定的格式将其格式化成字符串
        intermediate_steps = kwargs.pop('intermediate_steps')
        thoughts = ''
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f'\nObservation: {observation}\nThought: '

        # 将格式化好的内容存入agent_scratchpad对应的值中
        kwargs['agent_scratchpad'] = thoughts

        # 这里和LLM Agent示例中的内容有所不同
        # 这里会先执行tools_getter对应的get_tools方法，
        # 即通过输入的内容获取最匹配的几个工具
        # 而不是像LLM Agent示例那样使用全部的工具
        tools = self.tools_getter(kwargs['input'])
        # 将工具列表按照“名字:描述”的格式转换成字符串，并将其存入tools对应的值中
        kwargs['tools'] = '\n'.join(
            [f'{tool.name}: {tool.description}' for tool in tools]
        )
        # 创建工具名称列表字符串，并将其存入tool_names对应的值中
        kwargs['tool_names'] = ', '.join([tool.name for tool in tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    input_variables=['input', 'intermediate_steps'],
)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # 如果LLM的输出中有“Final Answer”，则表示当前任务已执行完毕
        # 需要返回AgentFinish
        if 'Final Answer:' in llm_output:
            return AgentFinish(
                return_values={
                    'output': llm_output.split('Final Answer:')[-1].strip()
                },
                log=llm_output,
            )
        # 解析动作和动作输入
        reg = (r'Action\s*\d*\s*:(.*?)\n'
               r'Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)')
        match = re.search(reg, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(
                f'Could not parse LLM output: `{llm_output}`')
        action = match.group(1).strip()
        action_input = match.group(2)

        # 返回后续需要执行的AgentAction，并设置要执行的工具和工具输入
        return AgentAction(
            tool=action,
            tool_input=action_input.strip(' ').strip('"'),
            log=llm_output
        )

output_parser = CustomOutputParser()

llm = ChatOpenAI(
    model_name='gpt-4-1106-preview',
    temperature=0
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

_input = '秦朝统一六国的时间'

# 先获取可以获取的与问题相关的工具列表
tools = get_tools(_input)
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=['\nObservation:'],
    allowed_tools=tool_names,
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)
agent_executor.invoke(_input)

## Auto-GPT Agent

In [ ]:
from langchain.agents import Tool
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_community.tools import ReadFileTool
from langchain_community.tools import WriteFileTool

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name='search',
        func=search.run,
        description='useful for when you need to '
                    'answer questions about current events. '
                    'You should ask targeted questions',
    ),
    WriteFileTool(),
    ReadFileTool(),
]
embedding = OpenAIEmbeddings()
vectorstore = Chroma(
    persist_directory='./chroma',
    embedding_function=embedding
)

agent = AutoGPT.from_llm_and_tools(
    ai_name='Tom',
    ai_role='Assistant',
    tools=tools,
    llm=ChatOpenAI(
        model_name='gpt-4o',
        temperature=0
    ),
    memory=vectorstore.as_retriever(),
)

# 打开详情模式
agent.chain.verbose = True

# 生成一个关于今天杭州天气的中文报告
agent.run([
    'Please help me write a Chinese report on '
    'the weather in Hangzhou today, in which the '
    'temperature unit needs to be in Celsius and '
    'saved as "hangzhou_weather.txt"'
])

# LangGraph

## 示例

In [ ]:
import os
import operator
import functools
from typing import Annotated, Sequence, TypedDict

from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

from langgraph.graph import StateGraph, END


@tool('web_search')
def web_search(query):
    """通过Google SERP API进行互联网搜索查询"""
    search = SerpAPIWrapper()
    return search.run(query)


@tool('red_book_writer')
def write_red_book(content):
    """根据一段内容，写一个小红书文案"""
    chat = ChatOpenAI(model='gpt-4-turbo-preview')
    system_msg = SystemMessage(
        content='你的任务是以小红书博主的文章结构，以我给出的主题，写一篇推荐帖。'
                '你的回答应包括使用表情符号来增加趣味和互动，'
                '以及与每个段落相匹配的图片。'
                '请以一个引人入胜的介绍开始，为你的推荐帖设置基调。'
                '然后，提供至少3个与主题相关的段落，突出它们的特点和吸引力。'
                '在你的写作中使用表情符号，使文案更加引人入胜和有趣。'
    )
    messages = [
        system_msg,
        HumanMessage(content=content),
    ]
    response = chat(messages)
    return response.content

system_prompt = (
    'You are a supervisor tasked with managing a conversation between the'
    ' following workers:  {members}. Given the following user request,'
    ' respond with the worker to act next. Each worker will perform a'
    ' task and respond with their results and status. When finished,'
    ' respond with FINISH.'
)

members = ['Search_Engine', 'Red_Book_Writer']
options = ['FINISH'] + members


# 使用OpenAI的函数调用，可以更方便地输出一个结构化的内容
func_structure = {
    'name': 'route',
    'description': 'Select the next role.',
    'parameters': {
        'title': 'routeSchema',
        'type': 'object',
        'properties': {
            'next': {
                'title': 'Next',
                'anyOf': [
                    {'enum': options},
                ],
            }
        },
        'required': ['next'],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder(variable_name='messages'),
        (
            'system',
            'Given the conversation above, who should act next?'
            ' Or should we FINISH? Select one of: {options}',
        ),
    ]
).partial(options=str(options), members=', '.join(members))

llm = ChatOpenAI(model='gpt-4-turbo-preview')
supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[func_structure], function_call='route')
    | JsonOutputFunctionsParser()
)

def create_agent(llm, tools, system_prompt):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                'system',
                system_prompt,
            ),
            MessagesPlaceholder(variable_name='messages'),
            MessagesPlaceholder(variable_name='agent_scratchpad'),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {'messages': [HumanMessage(content=result['output'], name=name)]}


search_engine_agent = create_agent(
    llm,
    [web_search],
    '你是一个网络搜索引擎'
)
search_engine_node = functools.partial(
    agent_node,
    agent=search_engine_agent,
    name='Search_Engine'
)

red_book_agent = create_agent(
    llm,
    [write_red_book],
    '你的主要职责是根据给定的内容编写小红书文案。'
)
red_book_node = functools.partial(
    agent_node,
    agent=red_book_agent,
    name='Red_Book_Writer'
)

# 状态对象存储了两个字段：
# messages用于保存上游处理完的信息
# next用于记录要执行哪个下游节点
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

workflow = StateGraph(AgentState)
workflow.add_node('Search_Engine', search_engine_node)
workflow.add_node('Red_Book_Writer', red_book_node)
workflow.add_node('supervisor', supervisor_chain)

# 这里实际上添加的代码逻辑是，执行完对应的节点再回到supervisor节点
for member in members:
    workflow.add_edge(member, 'supervisor')

conditional_map = {k: k for k in members}
conditional_map['FINISH'] = END
workflow.add_conditional_edges(
    'supervisor',
    lambda x: x['next'],
    conditional_map
)

# 设置起始节点
workflow.set_entry_point('supervisor')
graph = workflow.compile()

for s in graph.stream({
    'messages': [
        HumanMessage(
            content='请先搜索OpenAI的Sora，并根据搜索结果写一篇小红书文案'
        )
    ]
}):
    if '__end__' not in s:
        print(s)
        print('----')